In [ ]:
import global_align as ga

In [ ]:
def get_gap_penalty(gap_existence_cost:int, gap_extension_cost:int, gap_length:int) -> int:
    """Get the cost for a single gap in the alignment.
    
    Note that the gap_extension_cost only factors into the total gap
    penalty when gap_length > 1.

    https://en.wikipedia.org/wiki/Gap_penalty#Affine

    Args:
        gap_existence_cost: gap_existence_cost >= 0
        gap_extension_cost: gap_extension_cost >= 0
        gap_length: Includes the current gap. gap_length >= 1.
    """
    return gap_existence_cost + gap_extension_cost * (gap_length - 1)

In [ ]:
get_gap_penalty(
    gap_existence_cost=2,
    gap_extension_cost=1,
    gap_length=2
)

In [1]:
GAP_SCORE = -2 
GAP_EXISTENCE_COST = 0
scoring_mat = {
    "A": {"A": 1, "C": -1, "G": -1, "T": -1, "-": -2},
    "C": {"A": -1, "C": 1, "G": -1, "T": -1, "-": -2},
    "G": {"A": -1, "C": -1, "G": 1, "T": -1, "-": -2},
    "T": {"A": -1, "C": -1, "G": -1, "T": 1, "-": -2},
    "-": {"A": -2, "C": -2, "G": -2, "T": -2, "-": -2},
}

In [2]:
scoring_mat["C"]["C"]

1

In [3]:
seq_1 = "CCTTATCGTAC"
seq_2 = "GGAGCTATGGCCTC"

# 0: gap in seq_1
# 1: gap in seq_2
# 2: match/mismatch

In [4]:
m = len(seq_1)
n = len(seq_2)
F_mat_num_rows = m + 1
F_mat_num_cols = n + 1

In [5]:
# https://www.freecodecamp.org/news/list-within-a-list-in-python-initialize-a-nested-list/
partial_F_mat = [
    [0]*(F_mat_num_cols) for i in range(2)
]
partial_F_mat

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

We can go one row at a time through the F_mat (starting at row index 1, col index 1 and always skipping col index 0).  Simultaneously, fill out the best_paths_mat.  (Note that we need to save the entirety of the best_paths_mat.)  We only need to keep the col 0 of the F_mat and two rows in memory at a time.

In [6]:
# Take care of initialization with gap scores.
# Loop prep
j = 1
gap_score_cum_sum = 0

seq_2_index = j - 1
cur_gap_score = -GAP_EXISTENCE_COST + scoring_mat["-"][seq_2[seq_2_index]]
partial_F_mat[0][j] = cur_gap_score

for j in range(2, F_mat_num_cols):
    # Prep for this iteration
    seq_2_index = j - 1
    gap_score_cum_sum += cur_gap_score

    # body of loop
    cur_gap_score = cur_gap_score + scoring_mat["-"][seq_2[seq_2_index]]
    partial_F_mat[0][j] = cur_gap_score

partial_F_mat

[[0, -2, -4, -6, -8, -10, -12, -14, -16, -18, -20, -22, -24, -26, -28],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

In [ ]:
# Take care of initialization with gap scores.
# Loop prep
gap_length_cum_sum = 0
# gap_score_cum_sum = 0

if GAP_EXTENSION_COST == 0:
    modified_gap_ext_cost = 1

for j in range(1, F_mat_num_cols):
    # Prep for this iteration
    seq_2_index = j - 1
    gap_length_cum_sum += 1
    # prev_gap_score = partial_F_mat[0][j - 1]
    # print("gap_score_cum_sum")
    # print(gap_score_cum_sum)

    # loop body
    current_gap_penalty = get_gap_penalty(
        gap_existence_cost=-scoring_mat["-"][seq_2[seq_2_index]],
        gap_extension_cost=modified_gap_ext_cost,
        gap_length=gap_length_cum_sum
    )
    # print("current_gap_penalty")
    # print(current_gap_penalty)
    partial_F_mat[0][j] = - current_gap_penalty

partial_F_mat

In [ ]:
get_gap_penalty(
    gap_existence_cost=scoring_mat["-"][seq_2[seq_2_index]],
    gap_extension_cost=GAP_EXTENSION_COST,
    gap_length=50
)

In [ ]:
partial_F_mat[0] = [GAP_SCORE*x for x in range(F_mat_num_cols)]
partial_F_mat

In [ ]:
partial_F_mat[1][0] = GAP_SCORE

partial_F_mat

In [ ]:
best_paths_mat_num_rows = F_mat_num_rows
best_paths_mat_num_cols = F_mat_num_cols
best_paths_mat = [[0]*best_paths_mat_num_cols for i in range(best_paths_mat_num_rows)]

# Get 1's in the beginning of each row.
for i in range(1, best_paths_mat_num_rows):
    best_paths_mat[i][0] = 1

best_paths_mat

In [ ]:
# Pre loop
i = 1
partial_F_mat_prev_row_id = 0
partial_F_mat_cur_row_id = 1

for j in range(1, F_mat_num_cols):
    seq_1_index = i - 1
    seq_2_index = j - 1

    score_for_path_a = partial_F_mat[partial_F_mat_cur_row_id][j - 1]
    score_for_path_b = partial_F_mat[partial_F_mat_prev_row_id][j]
    score_for_path_c = partial_F_mat[partial_F_mat_prev_row_id][j - 1]

    score_for_gap_in_seq_1 = score_for_path_a + GAP_SCORE
    score_for_gap_in_seq_2 = score_for_path_b + GAP_SCORE
    score_for_match_mismatch = score_for_path_c + scoring_mat[seq_1[seq_1_index]][seq_2[seq_2_index]]
    
    possible_new_scores = [score_for_gap_in_seq_1, score_for_gap_in_seq_2, score_for_match_mismatch]
    max_possible_new_score = max(possible_new_scores)
    
    partial_F_mat[partial_F_mat_cur_row_id][j] = max_possible_new_score
    best_type_of_path = possible_new_scores.index(max_possible_new_score)
    # print(f"i: {i}      j: {j}")
    best_paths_mat[i][j] = best_type_of_path


In [ ]:
partial_F_mat

In [ ]:
for i in range(2, F_mat_num_rows):
    # Prep for a new row iteration.
    # https://stackoverflow.com/a/14836456
    # Do some swapping. 
    partial_F_mat_prev_row_id, partial_F_mat_cur_row_id = partial_F_mat_cur_row_id, partial_F_mat_prev_row_id
    # Update the 0th column based on how gaps are penalized.
    partial_F_mat[partial_F_mat_cur_row_id][0] = GAP_SCORE*i
    
    for j in range(1, F_mat_num_cols):
        seq_1_index = i - 1
        seq_2_index = j - 1

        score_for_path_a = partial_F_mat[partial_F_mat_cur_row_id][j - 1]
        score_for_path_b = partial_F_mat[partial_F_mat_prev_row_id][j]
        score_for_path_c = partial_F_mat[partial_F_mat_prev_row_id][j - 1]

        score_for_gap_in_seq_1 = score_for_path_a + GAP_SCORE
        score_for_gap_in_seq_2 = score_for_path_b + GAP_SCORE
        score_for_match_mismatch = score_for_path_c + scoring_mat[seq_1[seq_1_index]][seq_2[seq_2_index]]
        
        possible_new_scores = [score_for_gap_in_seq_1, score_for_gap_in_seq_2, score_for_match_mismatch]
        max_possible_new_score = max(possible_new_scores)
        
        partial_F_mat[partial_F_mat_cur_row_id][j] = max_possible_new_score
        best_type_of_path = possible_new_scores.index(max_possible_new_score)
        # print(f"i: {i}      j: {j}")
        best_paths_mat[i][j] = best_type_of_path

In [ ]:
score = max_possible_new_score
score

In [ ]:
partial_F_mat

In [ ]:
best_paths_mat

In [ ]:
# traceback
# Prepare for loop.
seq_1_aligned = []
seq_2_aligned = []
middle_part = []

num_alignment_moves = max(best_paths_mat_num_rows, best_paths_mat_num_cols) - 1

# Start at the bottom-right.
seq_1_index = m - 1
seq_2_index = n - 1

for w in range(num_alignment_moves):
    # Prep for this iteration.
    # Because of the initial row and column in
    # best_paths_mat that doesn't align with
    # any parts of the two sequence, the indices
    # are off by one.
    best_paths_mat_row_index = seq_1_index + 1
    best_paths_mat_col_index = seq_2_index + 1

    path_indicator = best_paths_mat[best_paths_mat_row_index][best_paths_mat_col_index]

    if path_indicator == 0:
        middle_part.append(" ")
        seq_1_aligned.append("-")
        seq_2_aligned.append(seq_2[seq_2_index])
        seq_2_index -= 1
    elif path_indicator == 1:
        middle_part.append(" ")
        seq_1_aligned.append(seq_1[seq_1_index])
        seq_1_index -= 1
        seq_2_aligned.append("-")
    else:
        seq_1_letter = seq_1[seq_1_index]
        seq_2_letter = seq_2[seq_2_index]
        if seq_1_letter == seq_2_letter:
            # There was a match.
            middle_part.append("|")
        else:
            # There was not a match.
            middle_part.append("*")

        seq_1_aligned.append(seq_1[seq_1_index])
        seq_1_index -= 1
        seq_2_aligned.append(seq_2[seq_2_index])
        seq_2_index -= 1

In [ ]:
seq_1_aligned.reverse()
middle_part.reverse()
seq_2_aligned.reverse()

In [ ]:
seq_1_aligned_out = "".join(seq_1_aligned)
middle_part_out = "".join(middle_part)
seq_2_aligned_out = "".join(seq_2_aligned)

In [ ]:
print(seq_1_aligned_out)
print(middle_part_out)
print(seq_2_aligned_out)
print(score)